In [1]:
!pip install pytorch_lightning==1.3.8

     |████████████████████████████████| 813 kB 13.9 MB/s 
     |████████████████████████████████| 829 kB 85.9 MB/s 
     |████████████████████████████████| 10.6 MB 59.9 MB/s 
     |████████████████████████████████| 118 kB 69.3 MB/s 
     |████████████████████████████████| 234 kB 68.7 MB/s 
     |████████████████████████████████| 636 kB 56.1 MB/s 
     |████████████████████████████████| 1.3 MB 55.8 MB/s 
     |████████████████████████████████| 294 kB 65.5 MB/s 
     |████████████████████████████████| 142 kB 80.0 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=6f2184ea4c1842b3b738344ab5637e89046f5bb62bd1f7301679bf7acd15d3f3
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Attempting uninsta

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import pytorch_lightning as pl

In [3]:
class Model(pl.LightningModule):
    def __init__(self):
        super(Model, self).__init__()

        self.avg_train_loss = 0.
        self.loss = nn.CrossEntropyLoss()
        self.lr = 1e-4
        

    def step(self, batch):
        
        inp, trg = batch
        self(inp, trg)
        loss = self.loss(inp[0], trg[0])

        return {"loss": loss}

    def training_step(self, batch, batch_idx):
        

        loss = self.step(batch)['loss']
        # print('training_step')
        return self.step(batch)
    
    def validation_step(self, batch, batch_idx):
        

        loss_valid = self.step(batch)['loss']
        
        # print('validation_step')
        return {"loss": loss_valid}



    def training_epoch_end(self, outputs):
        # print('training_epoch_end')
        self.avg_train_loss = torch.stack([x['loss'] for x in outputs]).mean()
    
    def validation_epoch_end(self, outputs):
        # print('validation_epoch_end')
        avg_valid_loss = torch.stack([x['loss'] for x in outputs]).mean()
        print(f'Train Loss: {self.avg_train_loss}')
        print(f'Valid Loss: {avg_valid_loss}')

    def forward(self, inp, trg):
      
     
        return None

    
    def configure_optimizers(self):
        # optim = torch.optim.Adam(self.parameters())
        return None

In [6]:

input1 = torch.randn(2, 3,5, requires_grad=True)
target1 = torch.empty(2, 3, dtype=torch.long).random_(5)


loss1 = nn.CrossEntropyLoss(reduction='mean')
output = loss1(input1.permute(0, 2, 1), target1)
print(f'Mean loss: {output}')

Mean loss: 1.9617977142333984


In [7]:
train_dataset = torch.utils.data.TensorDataset(input1, target1)
test_dataset = torch.utils.data.TensorDataset(input1, target1)

train_loader = torch.utils.data.DataLoader(
                        dataset=train_dataset,
                        batch_size=1,
                        shuffle=False
                        )


test_loader = torch.utils.data.DataLoader(
                        dataset=test_dataset,
                        batch_size=1,
                        shuffle=False # Not necessary!
                        )

In [9]:
model = Model()
trainer = pl.Trainer(max_epochs=2, progress_bar_refresh_rate=0, log_every_n_steps=1, num_sanity_val_steps=0)
trainer.fit(model, train_dataloader=train_loader, val_dataloaders=test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/optimizers.py:38: UserWarning: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer
  UserWarning,

  | Name | Type             | Params
------------------------------------------
0 | loss | CrossEntropyLoss | 0     
------------------------------------------
0         Trainable params
0         Non-trainable params
0         Total params
0.000     Total estimated model params size (MB)


Train Loss: 1.961797833442688
Valid Loss: 1.961797833442688
Train Loss: 1.961797833442688
Valid Loss: 1.961797833442688
